In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

import requests
from bs4 import BeautifulSoup

headers={'USer-Agent': 'Mozilla/5.0'}

response=requests.get(url, headers = headers)

r = requests.get(url)
html = r.text

soup = BeautifulSoup(html, "html.parser")

In [4]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  52.34 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  36.45 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  39.96 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  46.04 MB/s
Libraries imported.


In [5]:
table=soup.find('table',{'class':'wikitable sortable'})
print(len(table))

2


In [6]:
import csv
list_of_rows = []
for row in table.findAll('tr'):
    list_of_cells = []
    for cell in row.findAll('td'):
        text = cell.text
        list_of_cells.append(text)
    list_of_rows.append(list_of_cells)
# Print into .csv and add proper headers.
outfile = open("outputfile.csv", "w")
writer = csv.writer(outfile)
writer.writerow(['Postcode', 'Borough', 'Neighbourhood'])
writer.writerows(list_of_rows)
print("FINISHED")




FINISHED


In [7]:
!conda install -c conda-forge geopy --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  23.42 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  21.94 MB/s


In [8]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
neighbourhoods = pd.read_csv("outputfile.csv") 

In [15]:
neighbourhoods.head()

neighbourhoods['Neighbourhood'] = neighbourhoods['Neighbourhood'].str.strip('\n')
neighbourhoods.head()

newdf = neighbourhoods[neighbourhoods['Neighbourhood'] != 'Not assigned']
newdf.head(10)



,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern
14,M3B,North York,Don Mills North


In [13]:
newdf.shape

(183, 3)

In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighbourhoods['Borough'].unique()),
        neighbourhoods.shape[0]
    )
)


The dataframe has 12 boroughs and 240 neighborhoods.


In [36]:
!wget -q -O 'geospadata.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

geodata = pd.read_csv("geospadata.csv") 
geodata.sort_values(['Postal Code'])
result = pd.merge(neighbourhoods, geodata, left_on='Postcode', right_on='Postal Code')
result.drop(["Postcode"], axis=1)

result = result[['Postal Code','Borough','Neighbourhood','Latitude','Longitude']]
result['Neighbourhood'] = result['Neighbourhood'].str.strip('\n')

result.head()



Data downloaded!


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [11]:
from geopy.geocoders import Nominatim

In [18]:
#define a user_agent. We will name our agent toronto_explorer
geolocator = Nominatim(user_agent="toronto_explorer")
address = 'Toronto,Canada'
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [22]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [37]:
# add markers to map
import itertools

for lat, lng, borough, neighborhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighbourhood']):

    label = '{}, {}'.format(neighborhood, borough)

    label = folium.Popup(label, parse_html=True)

    folium.CircleMarker(

        [lat, lng],

        radius=5,

        popup=label,

        color='blue',

        fill=True,

        fill_color='#3186cc',

        fill_opacity=0.7,

        parse_html=False).add_to(map_toronto)  

    

map_toronto

In [101]:
#cluster only the neighborhoods in North York Borough
NorthYork_data = result[result['Borough'] == 'North York'].reset_index(drop=True)
NorthYork_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,Lawrence Heights,43.718518,-79.464763
3,M6A,North York,Lawrence Manor,43.718518,-79.464763
4,M3B,North York,Don Mills North,43.745906,-79.352188


In [49]:
#Let's get the geographical coordinates of  North York.

geolocator = Nominatim(user_agent="toronto_explorer")
address = 'North York, Toronto'
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of North York are {}, {}.'.format(latitude, longitude))


The geograpical coordinates of North York are 43.7708175, -79.4132998.


In [52]:
# create map of North York using latitude and longitude values

map_NorthYork = folium.Map(location=[latitude, longitude], zoom_start=11)

In [53]:
# add markers to map

for lat, lng, label in zip(NorthYork_data['Latitude'], NorthYork_data['Longitude'], NorthYork_data['Neighbourhood']):

    label = folium.Popup(label, parse_html=True)

    folium.CircleMarker(

        [lat, lng],

        radius=5,

        popup=label,

        color='blue',

        fill=True,

        fill_color='#3186cc',

        fill_opacity=0.7,

        parse_html=False).add_to(map_NorthYork)  

    

map_NorthYork

In [55]:
#Define Foursquare Credentials and Version

CLIENT_ID = 'GF5SIRQTLDNUNUROFXILZBGQYLNEEREB0YAMMAUU0BAGVDCL' # your Foursquare ID

CLIENT_SECRET = 'AP0BKKOI2ZZXYSROSCWUAIHZ1ZUDO0O2HZSLT30EQLB1TDH1' # your Foursquare Secret

VERSION = '20180605' # Foursquare API version


In [60]:
#Let's explore the first neighborhood in our dataframe.
NorthYork_data.loc[0, 'Neighbourhood']


'Parkwoods'

In [59]:
#Get the neighborhood's latitude and longitude values.
neighborhood_latitude = NorthYork_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = NorthYork_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = NorthYork_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [64]:
#Now, let's get the top 100 venues that are in Parkwoods within a radius of 500 meters.

#First, let's create the GET request URL. 


LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GF5SIRQTLDNUNUROFXILZBGQYLNEEREB0YAMMAUU0BAGVDCL&client_secret=AP0BKKOI2ZZXYSROSCWUAIHZ1ZUDO0O2HZSLT30EQLB1TDH1&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [65]:
urlresults = requests.get(url).json()

urlresults

{'meta': {'code': 200, 'requestId': '5c92935ef594df6af0abdb0e'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4e8d9dcdd5fbbbb6b3003c7b-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d163941735',
         'name': 'Park',
         'pluralName': 'Parks',
         'primary': True,
         'shortName': 'Park'}],
       'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'location': {'address': 'Toronto',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 245,
        'formattedAddress': ['Toronto', 'Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.751976046055574,
          'lng': -79.33214044722958}],
        'lat

In [66]:
# function that extracts the category of the venue

def get_category_type(row):

    try:

        categories_list = row['categories']

    except:

        categories_list = row['venue.categories']

        

    if len(categories_list) == 0:

        return None

    else:

        return categories_list[0]['name']

In [70]:
from pandas.io.json import json_normalize
venues = urlresults['response']['groups'][0]['items']

nearby_venues = json_normalize(venues)
# filter columns

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,KFC,Fast Food Restaurant,43.754387,-79.333021
2,Variety Store,Food & Drink Shop,43.751974,-79.333114
3,TTC stop - 44 Valley Woods,Bus Stop,43.755402,-79.333741


In [71]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [102]:
#create a function to repeat the same process to all the neighborhoods in NorthYork

def getNearbyVenues(names, latitudes, longitudes, radius=500):
  
    venues_list=[]

    for name, lat, lng in zip(names, latitudes, longitudes):

        print(name)
          

        # create the API request URL

        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(

            CLIENT_ID, 

            CLIENT_SECRET, 

            VERSION, 

            lat, 

            lng, 

            radius, 

            LIMIT)

            

        # make the GET request

        results = requests.get(url).json()["response"]['groups'][0]['items']

        

        # return only relevant information for each nearby venue

        venues_list.append([(

            name, 

            lat, 

            lng, 

            v['venue']['name'], 

            v['venue']['location']['lat'], 

            v['venue']['location']['lng'],  

            v['venue']['categories'][0]['name']) for v in results])



    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])

    nearby_venues.columns = ['Neighbourhood', 

                  'Neighborhood Latitude', 

                  'Neighborhood Longitude', 

                  'Venue', 

                  'Venue Latitude', 

                  'Venue Longitude', 

                  'Venue Category']

    

    return(nearby_venues)


#run the above function on each neighborhood and create a new dataframe called NorthYork_venues.



NorthYork_venues = getNearbyVenues(names=NorthYork_data['Neighbourhood'],

                                   latitudes=NorthYork_data['Latitude'],

                                   longitudes=NorthYork_data['Longitude']

                                  )


Parkwoods
Victoria Village
Lawrence Heights
Lawrence Manor
Don Mills North
Glencairn
Flemingdon Park
Don Mills South
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Fairview
Henry Farm
Oriole
Northwood Park
York University
Bayview Village
CFB Toronto
Downsview East
Silver Hills
York Mills
Downsview West
Maple Leaf Park
North Park
Upwood Park
Humber Summit
Newtonbrook
Willowdale
Downsview Central
Bedford Park
Lawrence Manor East
Emery
Humberlea
Willowdale South
Downsview Northwest
York Mills West
Willowdale West


In [76]:
print(NorthYork_venues.shape)

(469, 7)


In [77]:
NorthYork_venues.head()

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [78]:
# check how many venues were returned for each neighborhood
NorthYork_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Bathurst Manor,17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
Bedford Park,26,26,26,26,26,26
CFB Toronto,3,3,3,3,3,3
Don Mills North,4,4,4,4,4,4
Don Mills South,21,21,21,21,21,21
Downsview Central,2,2,2,2,2,2
Downsview East,3,3,3,3,3,3
Downsview North,17,17,17,17,17,17


In [80]:
#find out how many unique categories can be curated from all the returned venues

print('There are {} unique categories.'.format(len(NorthYork_venues['Venue Category'].unique())))

There are 106 unique categories.


In [103]:
#Analyze Each Neighborhood

# one hot encoding

NorthYork_onehot = pd.get_dummies(NorthYork_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe

NorthYork_onehot['Neighbourhood'] = NorthYork_venues['Neighbourhood'] 
# move neighborhood column to the first column

fixed_columns = [NorthYork_onehot.columns[-1]] + list(NorthYork_onehot.columns[:-1])

NorthYork_onehot = NorthYork_onehot[fixed_columns]

NorthYork_onehot.head(7)


,Neighbourhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lingerie Store,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Smoothie Shop,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [88]:
# print each neighborhood along with the top 5 most common venues
NorthYork_grouped = NorthYork_onehot.groupby('Neighbourhood').mean().reset_index()
num_top_venues = 5
for hood in NorthYork_grouped['Neighbourhood']:

    print("----"+hood+"----")

    temp = NorthYork_grouped[NorthYork_grouped['Neighbourhood'] == hood].T.reset_index()

    temp.columns = ['venue','freq']

    temp = temp.iloc[1:]

    temp['freq'] = temp['freq'].astype(float)

    temp = temp.round({'freq': 2})

    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))

    print('\n')
	

----Bathurst Manor----
         venue  freq
0  Coffee Shop  0.12
1  Supermarket  0.06
2        Diner  0.06
3     Pharmacy  0.06
4  Pizza Place  0.06


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park----
                  venue  freq
0    Italian Restaurant  0.12
1           Coffee Shop  0.08
2  Fast Food Restaurant  0.08
3               Butcher  0.04
4        Ice Cream Shop  0.04


----CFB Toronto----
            venue  freq
0         Airport  0.33
1            Park  0.33
2        Bus Stop  0.33
3  Lingerie Store  0.00
4   Movie Theater  0.00


----Don Mills North----
                  venue  freq
0  Gym / Fitness Center  0.25
1  Caribbean Restaurant  0.25
2                  Café  0.25
3   Japanese Restaurant  0.25
4                Lounge  0.00


----Don Mills South----
              venue  freq
0       Coffee Shop  0.1

In [93]:
#put that into a pandas dataframe

#write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):

    row_categories = row.iloc[1:]

    row_categories_sorted = row_categories.sort_values(ascending=False)

    return row_categories_sorted.index.values[0:num_top_venues]


#create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = NorthYork_grouped['Neighbourhood']

for ind in np.arange(NorthYork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NorthYork_grouped.iloc[ind, :], num_top_venues)
    
neighborhoods_venues_sorted.head()


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bathurst Manor,Coffee Shop,Frozen Yogurt Shop,Bank,Deli / Bodega,Restaurant,Diner,Sandwich Place,Shopping Mall,Pizza Place,Pharmacy
1,Bayview Village,Chinese Restaurant,Bank,Café,Japanese Restaurant,Frozen Yogurt Shop,Discount Store,Comfort Food Restaurant,Construction & Landscaping,General Entertainment,Convenience Store
2,Bedford Park,Italian Restaurant,Fast Food Restaurant,Coffee Shop,Greek Restaurant,Liquor Store,Pub,Pizza Place,Pharmacy,Butcher,Juice Bar
3,CFB Toronto,Airport,Bus Stop,Park,Women's Store,Discount Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
4,Don Mills North,Caribbean Restaurant,Gym / Fitness Center,Café,Japanese Restaurant,Women's Store,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop


In [111]:
# Cluster Neighborhoods
#Run k-means to cluster the neighborhood into 5 clusters.

kclusters = 5

NorthYork_grouped_clustering = NorthYork_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NorthYork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]



array([0, 0, 0, 3, 0, 0, 1, 3, 0, 0], dtype=int32)

In [115]:
NorthYork_merged = NorthYork_data
# merge NorthYork_grouped with toronto_data to add latitude/longitude for each neighborhood
NorthYork_merged = NorthYork_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

NorthYork_merged.head() 

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Park,Bus Stop,Food & Drink Shop,Fast Food Restaurant,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Coffee Shop,Pizza Place,Portuguese Restaurant,Hockey Arena,Intersection,Women's Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
2,M6A,North York,Lawrence Heights,43.718518,-79.464763,0.0,Clothing Store,Accessories Store,Athletics & Sports,Coffee Shop,Boutique,Event Space,Miscellaneous Shop,Vietnamese Restaurant,Furniture / Home Store,Fried Chicken Joint
3,M6A,North York,Lawrence Manor,43.718518,-79.464763,0.0,Clothing Store,Accessories Store,Athletics & Sports,Coffee Shop,Boutique,Event Space,Miscellaneous Shop,Vietnamese Restaurant,Furniture / Home Store,Fried Chicken Joint
4,M3B,North York,Don Mills North,43.745906,-79.352188,0.0,Caribbean Restaurant,Gym / Fitness Center,Café,Japanese Restaurant,Women's Store,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop


In [122]:
#visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(NorthYork_merged['Latitude'], NorthYork_merged['Longitude'], NorthYork_merged['Neighbourhood'], NorthYork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_color=rainbow,
        fill_opacity=0.7)
        
       
map_clusters

In [124]:
#Cluster 1

NorthYork_merged.loc[NorthYork_merged['Cluster Labels'] == 0, NorthYork_merged.columns[[1] + list(range(5, NorthYork_merged.shape[1]))]]

#Cluster 2

NorthYork_merged.loc[NorthYork_merged['Cluster Labels'] == 1, NorthYork_merged.columns[[1] + list(range(5, NorthYork_merged.shape[1]))]]

#Cluster 3

NorthYork_merged.loc[NorthYork_merged['Cluster Labels'] == 2, NorthYork_merged.columns[[1] + list(range(5, NorthYork_merged.shape[1]))]]

#Cluster 4

NorthYork_merged.loc[NorthYork_merged['Cluster Labels'] == 3, NorthYork_merged.columns[[1] + list(range(5, NorthYork_merged.shape[1]))]]

#Cluster 5

NorthYork_merged.loc[NorthYork_merged['Cluster Labels'] == 4, NorthYork_merged.columns[[1] + list(range(5, NorthYork_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,North York,4.0,Pizza Place,Women's Store,Discount Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
